In [42]:
import pandas as pd

df = pd.read_csv("./Training.csv")
n = len(df.columns)
symptom_columns = [col.lower().replace('_',' ').strip() for col in df.columns[:n-1]]
print(symptom_columns) 

for s in symptom_columns:
    if s=="headache": 
        print("x")

['itching', 'skin rash', 'nodal skin eruptions', 'continuous sneezing', 'shivering', 'chills', 'joint pain', 'stomach pain', 'acidity', 'ulcers on tongue', 'muscle wasting', 'vomiting', 'burning micturition', 'spotting  urination', 'fatigue', 'weight gain', 'anxiety', 'cold hands and feets', 'mood swings', 'weight loss', 'restlessness', 'lethargy', 'patches in throat', 'irregular sugar level', 'cough', 'high fever', 'sunken eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish skin', 'dark urine', 'nausea', 'loss of appetite', 'pain behind the eyes', 'back pain', 'constipation', 'abdominal pain', 'diarrhoea', 'mild fever', 'yellow urine', 'yellowing of eyes', 'acute liver failure', 'fluid overload', 'swelling of stomach', 'swelled lymph nodes', 'malaise', 'blurred and distorted vision', 'phlegm', 'throat irritation', 'redness of eyes', 'sinus pressure', 'runny nose', 'congestion', 'chest pain', 'weakness in limbs', 'fast heart rate', 'pain during bow

In [43]:
from thefuzz import process

def extract_symptoms_fuzzy(user_tokens, threshold=80):
    print(user_tokens)
    # user_tokens = ["fever", "headche"]
    print(user_tokens)
    matched_symptoms = []
    for token in user_tokens:
        match, score = process.extractOne(token, symptom_columns)
        if score > threshold:
            matched_symptoms.append(match)
    return list(set(matched_symptoms))

# user_text = "I have headache and fever"
# print(extract_symptoms_fuzzy(user_text))

In [44]:
# import spacy
# import scispacy

# nlp = spacy.load("en_core_sci_md")

# user_text = "I am feeling nauseous since the morning, also having stomachaches and headaches accompanied with it."

# user_text = user_text.lower()

# print(user_text)

# doc = nlp(user_text)
# print(doc.ents)

# l = []
# for ent in doc.ents:
#     print(ent.label)
#     l.append(ent.text)

# print(l)

# x = extract_symptoms_fuzzy(l)
# print(x)

In [45]:
# import spacy

# nlp = spacy.load("en_core_sci_md")

# def spacy_match_symptoms(user_text, threshold=0.72):
#     user_doc = nlp(user_text.lower())
    
#     matched_symptoms = []
#     for symptom in symptom_columns:
#         symptom_doc = nlp(symptom)
#         similarity = user_doc.similarity(symptom_doc)
#         if similarity > threshold:
#             matched_symptoms.append(symptom)
        

#     return list(set(matched_symptoms))

# # Example Usage
# user_text = "nauseous, headache, pain in lower abdomen, yellow skin, high fever, dry mouth"
# print(spacy_match_symptoms(user_text))

In [46]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
indices = []
vec = [0 for i in range(0, len(symptom_columns))]

symptom_embeddings = model.encode(symptom_columns)

def match_synonyms(user_text, threshold=0.7):
    user_embedding = model.encode(user_text)

    matched_symptoms = []
    for j in user_text.split(','):
        user_embedding = model.encode(j)
        for i, symptom in enumerate(symptom_columns):
            similarity = util.cos_sim(user_embedding, symptom_embeddings[i]).item()
            if similarity > threshold:
                matched_symptoms.append(symptom)
                print(i)
                indices.append(i)

    return matched_symptoms



user_text = "Frequent urination, excessive thirst, weight loss, fatigue, blurry vision"
print(match_synonyms(user_text))

sum = 0

for i in indices:
    vec[i] = 1

for i in vec:
    sum += i
print(sum)
print(vec)
print(len(vec))

19
14
49
['weight loss', 'fatigue', 'blurred and distorted vision']
3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
132


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

data = {
    "Disease": ["Flu", "Cold", "Malaria", "Diabetes", "Hypertension"],
    "Symptoms": [
        ["fever", "headache", "fatigue", "cough"],
        ["sneezing", "runny nose", "sore throat"],
        ["fever", "chills", "sweating", "nausea"],
        ["thirsty", "frequent urination", "weight loss"],
        ["headache", "dizziness", "fatigue"]
    ]
}
data = {
    "Disease": symptom_columns,
    "Symptoms":[
    ["Fever", "Headache", "Muscle pain", "Joint pain", "Rash", "Nausea"],
    ["Cough", "Shortness of breath", "Chest pain", "Fever", "Fatigue", "Chills"],
    ["Abdominal pain", "Diarrhea", "Vomiting", "Bloating", "Loss of appetite", "Weight loss"],
    ["Itchy skin", "Red eyes", "Runny nose", "Sneezing", "Congestion", "Wheezing"],
    ["Dizziness", "Sweating", "Palpitations", "Nausea", "Blurred vision", "Weakness"],
    ["Frequent urination", "Excessive thirst", "Weight loss", "Fatigue", "Blurry vision"],
    ["Sore throat", "Swollen lymph nodes", "Fever", "Body aches", "Loss of taste", "Cough"],
    ["Yellow skin", "Fatigue", "Dark urine", "Loss of appetite", "Nausea", "Swollen abdomen"],
    ["Joint pain", "Stiffness", "Swelling", "Fatigue", "Fever", "Reduced range of motion"],
    ["Chest pain", "Nausea", "Cold sweat", "Dizziness", "Shortness of breath", "Fatigue"]
]
}

df = pd.DataFrame(data)

# 2. Convert symptoms into binary features using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(df["Symptoms"])
y = df["Disease"]

# 3. Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# 5. Function to predict disease based on user input symptoms
def predict_disease(user_symptoms):
    user_input_vector = mlb.transform([user_symptoms])
    prediction = knn.predict(user_input_vector)
    return prediction[0]

# 6. Test the model
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# 7. Example User Input
user_symptoms = ["fever", "headache", "fatigue"]
predicted_disease = predict_disease(user_symptoms)
print(f"Predicted Disease: {predicted_disease}")